# Test Age and Gender Models on a Custom Image
This notebook loads your trained models and predicts age and gender for an image in the `test` folder.

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp

# Model and image config
IMG_SIZE = (128, 128)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_DIR = '../models'
TEST_IMG_DIR = '../test'
MAX_AGE = 116 

# Initialize MediaPipe Face Detector
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# List test images
test_images = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
print('Test images:', test_images)

In [ ]:
class BaseCNN(nn.Module):
    def __init__(self, output_units=1, task='age'):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(128),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * (IMG_SIZE[0] // 8) * (IMG_SIZE[1] // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, output_units)
        )
        self.task = task

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        if self.task == 'gender':
            x = torch.sigmoid(x)
        return x

In [ ]:
# Load the correct model class for ResNet-based models
from torchvision import models
import torch.nn as nn

class ResNetModel(nn.Module):
    def __init__(self, output_units=1, task='age'):
        super().__init__()
        self.resnet = models.resnet18(weights=None)  # weights=None for loading custom weights
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, output_units)
        self.task = task
    def forward(self, x):
        return self.resnet(x)

# Instantiate models
age_model = ResNetModel(output_units=1, task='age')
gender_model = ResNetModel(output_units=1, task='gender')

# Load weights
age_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'age_model_resnet.pth'), map_location=DEVICE))
gender_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'gender_model_resnet.pth'), map_location=DEVICE))

age_model.eval()
age_model.to(DEVICE)
gender_model.eval()
gender_model.to(DEVICE)

In [ ]:
# Block 4: Define transforms (should match validation/test transforms used in training)
test_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()
])

# Gender label mapping (edit if needed)
gender_map = {0: 'Female', 1: 'Male'}

# The rest of your code is correct, including the new face detection loop.

In [ ]:
print("\nRunning predictions with MediaPipe face detection...")

for fname in test_images:
    img_path = os.path.join(TEST_IMG_DIR, fname)
    
    # Read the image using OpenCV for MediaPipe compatibility
    img_cv2 = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe
    results = face_detection.process(img_rgb)
    
    # Check if any faces were detected
    if not results.detections:
        print(f"No face detected in {fname}. Skipping.")
        # Show the original image without any prediction
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(f'No face detected in {fname}')
        plt.show()
        continue

    # Get the bounding box of the first detected face
    detection = results.detections[0]
    bbox = detection.location_data.relative_bounding_box
    
    # Calculate pixel coordinates
    h, w, _ = img_cv2.shape
    x, y = int(bbox.xmin * w), int(bbox.ymin * h)
    box_w, box_h = int(bbox.width * w), int(bbox.height * h)

    # Crop the face from the original image
    cropped_face = img_rgb[y:y+box_h, x:x+box_w]

    # Convert to PIL Image and apply your transforms
    pil_face = Image.fromarray(cropped_face)
    img_tensor = test_transform(pil_face).unsqueeze(0).to(DEVICE)
    
    # Age prediction
    with torch.no_grad():
        age_pred_normalized = age_model(img_tensor).item()
        # Denormalize the age prediction back to years
        age_pred = round(age_pred_normalized * 116) 

    # Gender prediction
    with torch.no_grad():
        gender_pred = gender_model(img_tensor).item()
        gender_idx = int(min(max(round(gender_pred), 0), 1))
        gender_label = gender_map[gender_idx]

    # Draw the bounding box and display the result
    cv2.rectangle(img_cv2, (x, y), (x + box_w, y + box_h), (0, 255, 0), 2)
    cv2.putText(img_cv2, f'Age: {age_pred}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(img_cv2, f'Gender: {gender_label}', (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()